In [1]:
#Find person names <persName> in <body> that match the list  <listPerson> in the header. Then, fetch the person's xml:id and put the value into <persName> element in the <body>.
# eg. <persName ref="#Anna_Maria_van_Schurman">

# Create a dictionary with <persName> in the <body> as a key and use xml:id as a value

#Insert the value of xml:id into the corresponding persName element. eg. xml:id="Duke_Marlborough", <persName ref="#Duke_Marlborough"

#Latest file name Fossils1Ah_MASTER_DL24032021.xml on 24 March 2021

#To ignore digits from <add rend="del"> and <add rend="pencil"> 
#from <catnum>, remove: -; ____; ?; .; and a tick in front of a number.


#before output a new XML, ensure nothing is removed or deleted from the original XML.
#Last updated 24 March 2021
#Fossils corals xml

from lxml import etree
import re
import csv

In [2]:
path = '/Users/deborahleem/Documents/1_PhD/Fossils1Ah_MASTER_DL24032021.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()
root = mytree

In [3]:
doc.docinfo.encoding

'utf-8'

In [5]:
def remove_element(el):
    parent = el.getparent()
    if el.tail:
        prev = el.getprevious()
        if prev:
            prev.tail = (prev.tail or '') + el.tail
        else:
            parent.text = (parent.text or '') + el.tail
    parent.remove(el)

# Remove every <add rend="del"> and <add rend="pencil"> from document before the rest of analysis. Duke of Monmouth is the only one with deleted name.
#Confusing to have these with catnum
for e in list(root.iterfind('.//{*}add')):
    if ('rend' in e.attrib) and (e.attrib['rend'] == 'del' or e.attrib['rend'] == 'pencil'):
            continue
        remove_element(e)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 16)

In [ ]:
#which persName goes with which ID
#viaf map shows viaf number with the corresponding person name
# This has nothing to do with the body text
idmap = {}
viafmap = {}
# go through master list person names
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}listPerson/{http://www.tei-c.org/ns/1.0}person'):
    id = x.attrib['{http://www.w3.org/XML/1998/namespace}id']
    # do find all on x object for persName
    for name in x.findall('{http://www.tei-c.org/ns/1.0}persName'):
        text = name.text
        # create a mapping of name to person_id        
        idmap[text] = '#' + id
        idno = x.find("{*}idno")
        # if idno is found add to viafmap
        if idno is not None:
            viafmap[text] = idno.text


In [ ]:
#create mapping of persNames (keys) to personIDs (values) using the listperon in the header
#which persName goes with which personID
#viaf map shows viaf number with the corresponding person name
# This has nothing to do with the body text
variant_id_map = {}
# go through master list person names
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}listPerson/{http://www.tei-c.org/ns/1.0}person'):
    id = x.attrib['{http://www.w3.org/XML/1998/namespace}id']
    # do find all on x object for persName
    for name in x.findall('{http://www.tei-c.org/ns/1.0}persName'):
        text = name.text
        # create a mapping of name to person_id
#         check if name already exists. Add it to a list of available person_ids
        variant_id_map.setdefault(text, []).append(id)
#         variant_id_map[text] = '#' + id
# print(variant_id_map)
# with open('variant_id_map.csv')
import json
with open('variant_id_map.json', 'w') as fp:
    json.dump(variant_id_map, fp)

In [ ]:
len(variant_id_map)

In [ ]:
#This section is not necessary
def find_enclosing_div3(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div3':
            return element
        element = element.getparent()
    
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
#         print(element.tag)
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()
                
# for catnum in root.iterfind('.//{*}catnum'):
#     print(find_pb_xml_id(catnum))

In [ ]:
#This section is not necessary
def find_div2_label(element):
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}div2':
            label = element.find('{*}label')
            if label is not None:
                return ''.join(label.itertext()).strip()
            return
        element = element.getparent()

In [ ]:
#Clean up catnum
#This section is not necessary

def cleanup_catnum(text):
    #removing from beginning and end:
    # \s spaces/tabs/newlines
    # \- dashes
    # . dots
    # ? question marks
    # _ underscores
    # ✓
    text = re.sub(r'^[ ✓\t\n.\-_?]+', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+$', "", text)
    text = re.sub(r'[ ✓\t\n.\-_?]+\*', "*", text)
    text = text.replace('. ', ' ')
    return text

In [ ]:
#To clean up person name
def cleanup_person_name(name):
    # Remove all spaces at start
    name = re.sub(r'^\s+', "", name)
    # Get rid of all dots
    #name = re.sub(r'\.', '', name)
    # Get rid of final 's
    #name = re.sub('\s*\'s', '', name)
    # = means contnued on next line, remove that
    name = re.sub('\s*=[\s=]*', '', name)
    # Remove spaces in "M r" etc. when at beginning of any word
    name = re.sub(r'\bM r\b', 'Mr ', name)
    name = re.sub(r'\bM rs\b', 'Mrs ', name)
    name = re.sub(r'\bD r\b', 'Dr ', name)
    name = re.sub(r'\bS r\b', 'Sr ', name)
    name = re.sub(r'\bCap t\b', 'Capt ', name)
    name = re.sub(r'\bEsq r\b', 'Esqr ', name)
    name = re.sub(r'\bW m\b', 'Wm ', name)

    # Clean up multiple spaces again
    #name = re.sub(r'\s+', " ", name)
    
    return name

In [ ]:
#Page number
def find_pb_xml_id(element):
    element = find_enclosing_div3(element)
    while element is not None:
        if element.tag == '{http://www.tei-c.org/ns/1.0}pb':
            return element.attrib['{http://www.w3.org/XML/1998/namespace}id']
        element = element.getprevious()

In [ ]:
def lookup_name(name, x, catnums, page):
    #     Use the value from our dictionary variant_id_map
#     if 'duke' in name.lower():
#         import pdb; pdb.set_trace()  
    try:
        val = str(variant_id_map[name])
        val = val.replace('[', '').replace(']', '').replace('"', '').replace("'", '')
        x.set('ref', '#'+val)
        return True
    except:
        print('Issue adding in: ' + name + '\ncatnum: ' + str(catnums) + '\npage: ' + str(page) + '\n')

In [ ]:
path = '/Users/deborahleem/Documents/1_PhD/Fossils1Ah_MASTER_DL24032021.xml'
doc = etree.parse(open(path))
mytree = doc.getroot()

In [ ]:
def find_catnums_in_div3(div3):
    return [cleanup_catnum(''.join(cn.itertext())) for cn in div3.findall("{*}catnum")]

#matching names and inserting ID into persName ref
matches = {False: 0, True: 0}

data = []
# This is main body
for x in mytree.findall('.//{http://www.tei-c.org/ns/1.0}div3//{http://www.tei-c.org/ns/1.0}persName'):
    # All text inside persName tag
    name = re.sub(r'\s+', " ", ''.join(x.itertext()))
    name = cleanup_person_name(name)
    
    # A few more rules
#     if name not in idmap:
#         print("not match", name)

#     if name in idmap:
    div3 = find_enclosing_div3(x)
    div3_text = ''.join(div3.itertext()).strip()
    catnums = find_catnums_in_div3(div3)
    label = find_div2_label(x)
    if label.startswith('Index'):
        continue
    page = find_pb_xml_id(x)
#etree.tounicode(div3) to get XML
    # Modification to add in new attr to xml based on new requirement
    if lookup_name(name, x, catnums, page):
        row = [name, idmap.get(name, None), viafmap.get(name, None), catnums, label, page, div3_text, etree.tounicode(div3)]
        data.append(row)

#         matches[name in idmap] += 1
#     if name in idmap:
#         x.attrib['ref'] = idmap[name]
        
#print matches and count non-matches    
# print(matches)

'''
people_match_nonmatch_text.csv' gives the names that match from the header inc normalised names
and also the ones do not have their names in the header
'''
with open('people_match_nonmatch_text_after_attr2.csv', 'w') as file:
    writer = csv.writer(file)
    for row in data:
        writer.writerow(row)

In [ ]:
# save down modified xml to new filename
# mydata = etree.tostring(mytree)
# myfile = open("modified_misc.xml", "w")
# myfile.write(str(mydata))

doc.write("unicode_modified_misc5.xml", encoding='utf-8')

In [ ]:
!pwd

In [ ]:
dir(doc)
type(doc)

In [ ]:
# etree.tostring(doc, encoding='utf-8', xml_declaration=True)